In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd
from datetime import datetime

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
y_pred.std()

5.28140357655334

In [16]:
df_result = pd.DataFrame(index=df.index)
year = datetime.today().year
month = datetime.today().month
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['ride_duration'] = y_pred

In [17]:
df_result.to_parquet(
    f'./ride_prediction_duration_{year}_{month}.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)